# Advent of code 2024
## Challenge 6

## Part 1
### https://adventofcode.com/2024/day/6

This part of the challenge was quite alright. I could figure out all of the steps that I needed to take and I managed to implement them.

In [1]:
# This function is what changes the direction 90 degrees to the right every time
# the guard hits an obstacle
def turn(direction = []):
    if (direction == [-1,0]):
        return [0,1]
    elif (direction == [0,1]):
        return [1,0]
    elif (direction == [1,0]):
        return [0,-1]
    elif (direction == [0,-1]):
        return [-1,0]

In [2]:
# Reading the data input file
input_file = open("challenge_6_input.txt", "r")

line_of_characters = []
list_of_lines = []
total_occurences = 0
position = []
direction = [-1,0]

# Extract the input data into a list of characters
for line in input_file:
    line_of_characters = list(line.strip())
    list_of_lines.append(line_of_characters)
        
# First step is to find the starting point
# This is a double loop and the double loop is broken as soon as the
# starting position is found
# the starting position is stored as well to move the guard
for main_list_index, line in enumerate(list_of_lines):
    if ('^' in line):
        for nested_list_index, value in enumerate(line):
            if (value == '^'):
                position.append(main_list_index)
                position.append(nested_list_index)
                break
        else:
            continue
        break

In [ ]:
# Per the problem, the starting position counts as a position visit
# So the start count is one and an X is placed on the starting position
# placing the X is important to make sure the position is not counted again
# if the guard walks on it again
list_of_lines[position[0]][position[1]] = 'X'
position_count = 1

# This while loop runs as long as the guard reaches one of the edges of the map
while position[0] > 0 and position[0] < (len(list_of_lines) - 1) and position[1] > 0 and position[1] < (len(list_of_lines[0]) - 1):
        # If the next step of the guard in his direction is an obstacle, he turns
        if (list_of_lines[position[0] + direction[0]][position[1] + direction[1]] == '#'):
            direction = turn(direction)
        # If the position ahead has already been stepped on, the guard still takes the step, but the position is
        # not counted
        elif (list_of_lines[position[0] + direction[0]][position[1] + direction[1]] == 'X'):
            position[0] += direction[0]
            position[1] += direction[1]
        # If it is a new position ahead, the step is taken, 1 is added to the position count and the
        # position is marked as being stepped on
        elif (list_of_lines[position[0] + direction[0]][position[1] + direction[1]] == '.'):
            position[0] += direction[0]
            position[1] += direction[1]
            position_count += 1
            list_of_lines[position[0]][position[1]] = 'X'
            
print(position_count)

## Part 2

I struggled for this part of the challenge. My first reflex was to think about what is solution 1. I thought it reasonable and effective. But somehow I was never arriving at the right answer. And I read the problem many times. At some point I did figure out that I had to find positions. But I was still getting the wrong answer. I resigned myself to look for help online and I read that my solution was wrong because my algorith was placing obstacles on positions on which that guard had already walked on earlier. This invalidated the position, because the guard would have bumped into it earlier, and so the obstacle would not have the expected effect.

When I was reading, I fell upon a lot of comments mentioning that they "brute forced" the challenge. And this intrigued me, how did they brute force the challenge? How could it actually be brute forced? And then I had a breakthrough, which is solution 2. And so because I asked for a hint, I told myself that I would implement this brute force solution, which I did.

### Solution 1

In [4]:
# This library is used to make a complete copy of a list of nested lists. Because the built-in copy method of a list does
# not make copies of the nested lists. It makes references
import copy

In [5]:
# This function checks if the guard falls into a loop at any point during his walk. 
# That is until he enters falls into a loop or ends up stepping on an edge of the zone
def check_for_a_loop(position = [], direction = [], zone = []):
    # The node dictionnary stores two things, the position of an obstacle and the direction in which it is 
    # bumped. These two elements are a node. It is the key of the dictionnary. The value is a number: 1, 2, 3. Etc.
    # This value indicates the amount of time the node is traversed. If a node is traversed 2 times, we have a loop.
    node_dictionnary = {}
    
    # The loop runs for as long as the guard either enters the loop or steps on an edge of the zone.
    # each iteration of the loop is a step.
    while position[0] > 0 and position[0] < (len(zone) - 1) and position[1] > 0 and position[1] < (len(zone[0]) - 1):
        # If the step ahead is an obstacle, which in this part of the challenge is either # or O
        # This mean a node is traversed. This triggers a check to see if the guard entered in a loop
        if (zone[position[0] + direction[0]][position[1] + direction[1]] == '#' or zone[position[0] + direction[0]][position[1] + direction[1]] == 'O'):
            # We first create the key, which is a tuple of two tuples: the position of the obstacle, and the direction in which
            # it was bumped
            direction_tuple = tuple([tuple(position),tuple(direction)])
            # If the tuple is not in the dictionnary yet, it is added and given a value of 1
            if (direction_tuple not in node_dictionnary):
                node_dictionnary[direction_tuple] = 1
            else:
                # If it is already there, 1 is added
                node_dictionnary[direction_tuple] += 1
            # Then, lastly, if the obstacle is bumped from the same direction twice, we have a loop
            # and the function returns True
            if (node_dictionnary[direction_tuple] == 2):
                return True
            # To continue the walk of the guard, we make it turn 90 degrees after bumping an obstacle
            direction = turn(direction)
        # If the step ahead is not an obstacle, and is a position that has already been walked on,
        # we make the guard step on it without marking the position
        elif (zone[position[0] + direction[0]][position[1] + direction[1]] == 'X'):
            position[0] += direction[0]
            position[1] += direction[1]
        # If the step ahead is not an obstacle and has not been walked on yet, it is stepped on and 
        # marked as walked on
        elif (zone[position[0] + direction[0]][position[1] + direction[1]] == '.'):
            position[0] += direction[0]
            position[1] += direction[1]
            zone[position[0]][position[1]] = 'X'
    return False

In [6]:
# This function checks if there is an obstacle somewhere to the right of current position of the guard
# based on its current direction, the function is used to see if it is worth setting up an obstacle 
# at the next step of the guard to cause a bump. It is only worth doing so if it would make him bump into
# another obstacle

# This function could be improved with recursion, to see if a loop would be created without making the guard
# walk in the map with the created obstacle. So that if there is an obstacle 90 degrees to the right, the function
# checks if there is another obstacle 90 degrees to the right of the other obstacle, and if the count is 4, then we have
# a loop, otherwise we don't have one.
def check_for_neighboring_obstacle(position = [], direction = [], zone = []):
    # If the guard is currently going up, we look at the right of his current position
    if (direction == [-1,0]):
        for value in zone[position[0]][position[1] + 1:]:
            if(value == '#'):
                # If an obstacle is found, we return True, which gives the signal that we can put the 
                # new obstacle at the next step of the guard
                return True
        # If the are no obstacles, no obstacle will be placed on the next step of the guard, as we know it would
        # not lead to a loop
        return False
    # If the guard is going to the right, we look down from its current position.
    elif (direction == [0,1]):
        for value in zone[position[0] + 1:]:
            if(value[position[1]] == '#'):
                return True
        return False
    # This is a bit trickier, if the guard is going up, we look above its current position
    # However, the normal direction for iterating through list is from the top down.
    # in order to look for obstacles from the bottom to the top, the list of positions above
    # the guard is reversed
    elif (direction == [1,0]):
        reversed_list = zone[position[0]][:position[1]]
        reversed_list.reverse()
        for value in reversed_list:
            if(value == '#'):
                return True
        return False
    # It is the same principle when the guard is going down, we then have to look at his left, and so reverse the list
    # to iterate from the step closest to the guard
    elif (direction == [0,-1]):
        reversed_list = zone[:position[0]]
        reversed_list.reverse()
        for value in reversed_list:
            if(value[position[1]] == '#'):
                return True
        return False

In [ ]:
# This piece of code works in the same way as part 1 of the challenge, however, at every step, we check
# if there is an obstacle 90 degrees to the right of the current direction of the guard, if an obstacle was
# never placed at the position ahead of the guard before, if the position ahead is not the starting position
# if the position ahead has not been stepped on before. If all those conditions are met, we check if placing an
# obstacle at the next step of the guard would make him enter a loop. And all such positions.
input_file = open("challenge_6_input.txt", "r")

line_of_characters = []
list_of_lines = []
list_of_lines_copy = []
starting_position = []
position = []
direction = [-1,0]
list_of_obstruction_position = []

for line in input_file:
    line_of_characters = list(line.strip())
    list_of_lines.append(line_of_characters)
        
for main_list_index, line in enumerate(list_of_lines):
    if ('^' in line):
        for nested_list_index, value in enumerate(line):
            if (value == '^'):
                position.append(main_list_index)
                position.append(nested_list_index)
                break
        else:
            continue
        break

starting_position = position.copy()
list_of_lines[position[0]][position[1]] = 'X'

while position[0] > 0 and position[0] < (len(list_of_lines) - 1) and position[1] > 0 and position[1] < (len(list_of_lines[0]) - 1):
        
        if (list_of_lines[position[0] + direction[0]][position[1] + direction[1]] == '#'):
            direction = turn(direction)
        else:
            # If the position does not have an obstacle already, all the checks mentioned above are made in this if statement
            if (check_for_neighboring_obstacle(position,direction,list_of_lines) == True and [position[0] + direction[0], position[1] + direction[1]] != starting_position and tuple([position[0] + direction[0], position[1] + direction[1]]) not in list_of_obstruction_position and list_of_lines[position[0] + direction[0]][position[1] + direction[1]] != 'X'):
                # we create a deep coopy of the map, the list of lists. Using the imported package
                list_of_lines_copy = copy.deepcopy(list_of_lines)
                # we place the new obstacle and the next step of the guard, which will cause him to turn at the first
                # iteration of the function checking for a loop. The obstacle is added on the copy of the map
                # the main map is traversed as if no new obstacle was added.
                list_of_lines_copy[position[0] + direction[0]][position[1] + direction[1]] = 'O'
                # If a loop is created, the position of the added obstacle is added to a list, to avoid having duplicates
                # because we look for the unique positions
                if check_for_a_loop(position.copy(),direction.copy(), list_of_lines_copy) == True:
                    list_of_obstruction_position.append(tuple([position[0] + direction[0], position[1] + direction[1]]))
            # We keep walking on the main map
            position[0] += direction[0]
            position[1] += direction[1]
            # And marking the steps we take to make sure an obstacle cannot be placed on a position on which we already
            # walked
            list_of_lines[position[0]][position[1]] = 'X'

# the length of the unique positions list gives the amount of such positions
print(len(list_of_obstruction_position))

### Solution 2

In [ ]:
# This is a "brute force" solution
# It iterates through every single position of the map, even those on which the guard never steps on
# in his normal walk. At every position, we had an obstacle and have the guard start walking from his starting
# and see if a loop is created. If it is, 1 is added to the total amount of positions on which a loop would be
# created.
input_file = open("challenge_6_input.txt", "r")

line_of_characters = []
list_of_lines = []
list_of_lines_copy = []
starting_position = []
total_obstruction_positions = 0
direction = [-1,0]

for line in input_file:
    line_of_characters = list(line.strip())
    list_of_lines.append(line_of_characters)
        
for main_list_index, line in enumerate(list_of_lines):
    if ('^' in line):
        for nested_list_index, value in enumerate(line):
            if (value == '^'):
                starting_position.append(main_list_index)
                starting_position.append(nested_list_index)
                break
        else:
            continue
        break

list_of_lines[starting_position[0]][starting_position[1]] = 'X'

for index_main_list,line in enumerate(list_of_lines):
    for index_nested_list,spot in enumerate(line):
        # An obstacle is placed if there are no obstacles already and if the position is not the starting position of the guard
        if (list_of_lines[index_main_list][index_nested_list] != '#' and [index_main_list,index_nested_list] != starting_position):
            list_of_lines_copy = copy.deepcopy(list_of_lines)
            list_of_lines_copy[index_main_list][index_nested_list] = 'O'
            if check_for_a_loop(starting_position.copy(),direction.copy(), list_of_lines_copy) == True:
                total_obstruction_positions += 1
                print(total_obstruction_positions)
            
print(total_obstruction_positions)